In [1]:
#Monitoramento Vitor Luiz
import pandas as pd
from dash import Dash, html, dcc, callback, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import dash_daq as daq
from dash.dependencies import Input, Output, State
import plotly.io as pio
import dash
import dash_bootstrap_components as dbc
from datetime import datetime, timezone
from pytz import timezone
from plotly.subplots import make_subplots
import oficial_func as of

In [2]:
# HEX COLORS
# https://htmlcolorcodes.com
teste = "#9C1FBD"
teste1 = "#f70d1a"
teste2 = "#EEEE0A"
teste3 = "#3DEC0A"

In [4]:
df_UFF = of.importa_df("https://thingspeak.com/channels/1670099/")
df_UFF

created_at    object
entry_id       int64
field1         int64
field2         int64
field3         int64
field4         int64
field5         int64
field6         int64
field7         int64
dtype: object
<bound method NDFrame.describe of                  created_at  entry_id  field1  field2  field3  field4  field5  \
0   2023-02-07 15:04:14 UTC     23084      -1      -1      -1    3653     164   
1   2023-02-07 15:24:13 UTC     23085      -1      -1      -1    3653     158   
2   2023-02-07 15:34:12 UTC     23086      -1      -1      -1    3653     166   
3   2023-02-07 15:54:10 UTC     23087      -1      -1      -1    3653     166   
4   2023-02-07 16:14:08 UTC     23088      -1      -1      -1    3653     160   
..                      ...       ...     ...     ...     ...     ...     ...   
95  2023-02-08 12:42:17 UTC     23179      -1      -1      -1    3653     113   
96  2023-02-08 12:52:16 UTC     23180      -1      -1      -1    3653     105   
97  2023-02-08 13:02:15 UTC     23

,Data,entry_id,Ax,Ay,Az,Temperatura,Umidade1,Umidade2,Umidade3
Data,,,,,,,,,
2023-02-07 12:04:14-03:00,2023-02-07 15:04:14+00:00,23084,-1,-1,-1,3653,164,330,114
2023-02-07 12:24:13-03:00,2023-02-07 15:24:13+00:00,23085,-1,-1,-1,3653,158,328,117
2023-02-07 12:34:12-03:00,2023-02-07 15:34:12+00:00,23086,-1,-1,-1,3653,166,328,119
2023-02-07 12:54:10-03:00,2023-02-07 15:54:10+00:00,23087,-1,-1,-1,3653,166,327,115
2023-02-07 13:14:08-03:00,2023-02-07 16:14:08+00:00,23088,-1,-1,-1,3653,160,326,116
...,...,...,...,...,...,...,...,...,...
2023-02-08 09:42:17-03:00,2023-02-08 12:42:17+00:00,23179,-1,-1,-1,3653,113,317,96
2023-02-08 09:52:16-03:00,2023-02-08 12:52:16+00:00,23180,-1,-1,-1,3653,105,317,95
2023-02-08 10:02:15-03:00,2023-02-08 13:02:15+00:00,23181,-1,-1,-1,3653,105,317,91


In [5]:
#Candidatos a parâmetros da função
n1_u3 = 30
n2_u3 = 10
diferença_u3 = -10
col_u3 = "Umidade3"

In [6]:
ultima_info = df_UFF.tail(1)
print(ultima_info)
ultima_info.describe
ultima_info

                                               Data  entry_id  Ax  Ay  Az  \
Data                                                                        
2023-02-08 10:32:13-03:00 2023-02-08 13:32:13+00:00     23183  -1  -1  -1   

                           Temperatura  Umidade1  Umidade2  Umidade3  
Data                                                                  
2023-02-08 10:32:13-03:00         3653       107       316        92  


,Data,entry_id,Ax,Ay,Az,Temperatura,Umidade1,Umidade2,Umidade3
Data,,,,,,,,,
2023-02-08 10:32:13-03:00,2023-02-08 13:32:13+00:00,23183,-1,-1,-1,3653,107,316,92


In [7]:
ultima_info[col_u3]
ultima_info.loc[ultima_info[col_u3] < 50, "cor"] = "#9C1FBD" #roxo
ultima_info.loc[((ultima_info[col_u3] >= 50) & (ultima_info[col_u3] <= 100)), "cor"] = "#f70d1a" #vermelho
ultima_info.loc[((ultima_info[col_u3] > 100) & (ultima_info[col_u3] <= 200)), "cor"] = "#EEEE0A" #amarelo
ultima_info.loc[ultima_info[col_u3] > 200, "cor"] = "#3DEC0A" #verde
print(ultima_info[col_u3])
ultima_info["cor"]

Data
2023-02-08 10:32:13-03:00    92
Name: Umidade3, dtype: int64


C:\Users\DELL\AppData\Local\Temp\ipykernel_14120\1789178163.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ultima_info.loc[ultima_info[col_u3] < 50, "cor"] = "#9C1FBD" #roxo


Data
2023-02-08 10:32:13-03:00    #f70d1a
Name: cor, dtype: object

In [8]:
valor = ultima_info.iloc[0]["cor"]
valor

'#f70d1a'

In [9]:
media_n1 = df_UFF[col_u3].tail(n1_u3).mean()
media_n2 = df_UFF[col_u3].tail(n2_u3).mean()

print(media_n1)
print(media_n2)

media_n2 - media_n1

91.26666666666667
93.6


2.3333333333333286

In [10]:
#Se a subtração das médias derem negativo significa que o valor de U3 está diminuindo e logo está piorando a situação
if ((media_n2 - media_n1) <= diferença_u3):
    if valor == "#3DEC0A": #Quando verde ele vai para amarelo
        valor = "#EEEE0A"
    elif valor == "#EEEE0A": #Quando amarelo vai para vermelho
        valor = "#f70d1a"
    elif valor == "#f70d1a": #Quando vermelho vai para roxo
        valor = "#9C1FBD"
    else:
        valor = "#9C1FBD"

valor

'#f70d1a'